In [ ]:
# Install pyfuzzy-toolbox
!pip install pyfuzzy-toolbox -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import fuzzy_systems as fs
from fuzzy_systems import MamdaniSystem

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['figure.dpi'] = 100

In [ ]:
# ============================================================================
# Create Mamdani System
# ============================================================================
system = MamdaniSystem()

# ============================================================================
# INPUT 1: Service Quality (0 to 10)
# ============================================================================
system.add_input('service', (0, 10))
system.add_term('service', 'poor', 'triangular', (0, 0, 5))
system.add_term('service', 'acceptable', 'triangular', (0, 5, 10))
system.add_term('service', 'excellent', 'triangular', (5, 10, 10))

# ============================================================================
# INPUT 2: Food Quality (0 to 10)
# ============================================================================
system.add_input('food', (0, 10))
system.add_term('food', 'poor', 'trapezoidal', (0, 0, 3, 5))
system.add_term('food', 'acceptable', 'triangular', (3, 5, 7))
system.add_term('food', 'delicious', 'trapezoidal', (5, 7, 10, 10))

# ============================================================================
# OUTPUT: Tip Percentage (0 to 25%)
# ============================================================================
system.add_output('tip', (0, 25))
system.add_term('tip', 'low', 'triangular', (0, 0, 13))
system.add_term('tip', 'medium', 'triangular', (0, 13, 25))
system.add_term('tip', 'high', 'triangular', (13, 25, 25))

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 14))

# ============================================================================
# Subplot 1: Service Quality
# ============================================================================
x_service = np.linspace(0, 10, 200)
for term_name, term in system.input_variables['service'].terms.items():
    mu = term.membership(x_service)
    axes[0].plot(x_service, mu, label=term_name, linewidth=2)

axes[0].set_title('Membership Functions: Service Quality', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Service Quality (0-10)', fontsize=12)
axes[0].set_ylabel('Membership Degree', fontsize=12)
axes[0].legend(loc='upper right', fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.05, 1.05])

# ============================================================================
# Subplot 2: Food Quality
# ============================================================================
x_food = np.linspace(0, 10, 200)
for term_name, term in system.input_variables['food'].terms.items():
    mu = term.membership(x_food)
    axes[1].plot(x_food, mu, label=term_name, linewidth=2)

axes[1].set_title('Membership Functions: Food Quality', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Food Quality (0-10)', fontsize=12)
axes[1].set_ylabel('Membership Degree', fontsize=12)
axes[1].legend(loc='upper right', fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([-0.05, 1.05])

# ============================================================================
# Subplot 3: Tip
# ============================================================================
x_tip = np.linspace(0, 25, 200)
for term_name, term in system.output_variables['tip'].terms.items():
    mu = term.membership(x_tip)
    axes[2].plot(x_tip, mu, label=term_name, linewidth=2)

axes[2].set_title('Membership Functions: Tip', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Tip (%)', fontsize=12)
axes[2].set_ylabel('Membership Degree', fontsize=12)
axes[2].legend(loc='upper right', fontsize=11)
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim([-0.05, 1.05])

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# Define Fuzzy Rules
# ============================================================================
system.add_rules([
    {'service': 'poor', 'food': 'poor', 'tip': 'low', 'operator': 'AND'},
    {'service': 'poor', 'food': 'acceptable', 'tip': 'low'},
    {'service': 'acceptable', 'food': 'poor', 'tip': 'low'},
    {'service': 'acceptable', 'food': 'acceptable', 'tip': 'medium', 'operator': 'AND'},
    {'service': 'excellent', 'food': 'acceptable', 'tip': 'high'},
    {'service': 'acceptable', 'food': 'delicious', 'tip': 'high'},
    {'service': 'excellent', 'food': 'delicious', 'tip': 'high', 'operator': 'OR'}
])

In [ ]:
print(f"{'Service':<10} {'Food':<10} {'Tip':<12} {'Scenario'}")
print("-" * 80)

test_cases = [
    (2, 2, "Terrible experience"),
    (5, 5, "Average experience"),
    (9, 9, "Excellent experience"),
    (2, 9, "Poor service, excellent food"),
    (9, 2, "Excellent service, poor food"),
    (6.5, 8, "Good service, great food"),
]

for service_val, food_val, description in test_cases:
    result = system.evaluate([service_val, food_val])
    tip_calc = result['tip']
    print(f"{service_val:<10.1f} {food_val:<10.1f} {tip_calc:<12.2f} {description}")

In [ ]:
service_example = 6.5
food_example = 8.0

result = system.evaluate_detailed({'service': service_example, 'food': food_example})
tip_example = result['outputs']['tip']

fig, axes = plt.subplots(3, 1, figsize=(12, 14))

# ============================================================================
# Subplot 1: Service
# ============================================================================
x_service = np.linspace(0, 10, 200)
for term_name, term in system.input_variables['service'].terms.items():
    mu = term.membership(x_service)
    axes[0].plot(x_service, mu, label=term_name, linewidth=2)

axes[0].axvline(x=service_example, color='red', linestyle='--', linewidth=2,
                label=f'Value = {service_example}')
axes[0].set_title(f'Fuzzification: Service = {service_example}', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Service Quality (0-10)', fontsize=12)
axes[0].set_ylabel('Membership Degree', fontsize=12)
axes[0].legend(loc='upper right', fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.05, 1.05])

# ============================================================================
# Subplot 2: Food
# ============================================================================
x_food = np.linspace(0, 10, 200)
for term_name, term in system.input_variables['food'].terms.items():
    mu = term.membership(x_food)
    axes[1].plot(x_food, mu, label=term_name, linewidth=2)

axes[1].axvline(x=food_example, color='red', linestyle='--', linewidth=2,
                label=f'Value = {food_example}')
axes[1].set_title(f'Fuzzification: Food = {food_example}', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Food Quality (0-10)', fontsize=12)
axes[1].set_ylabel('Membership Degree', fontsize=12)
axes[1].legend(loc='upper right', fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([-0.05, 1.05])

# ============================================================================
# Subplot 3: Tip (aggregated and defuzzified output)
# ============================================================================
x_tip = np.linspace(0, 25, 200)
for term_name, term in system.output_variables['tip'].terms.items():
    mu = term.membership(x_tip)
    axes[2].plot(x_tip, mu, label=term_name, linewidth=2)

axes[2].axvline(x=tip_example, color='red', linestyle='--', linewidth=3,
                label=f'Tip = {tip_example:.2f}%')
axes[2].set_title('Aggregation and Defuzzification', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Tip (%)', fontsize=12)
axes[2].set_ylabel('Membership Degree', fontsize=12)
axes[2].legend(loc='upper right', fontsize=11)
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim([-0.05, 1.05])

plt.tight_layout()
plt.show()

In [ ]:
service_range = np.linspace(0, 10, 40)
food_range = np.linspace(0, 10, 40)
SERVICE, FOOD = np.meshgrid(service_range, food_range)

TIP = np.zeros_like(SERVICE)
for i in range(SERVICE.shape[0]):
    for j in range(SERVICE.shape[1]):
        try:
            result = system.evaluate({'service': SERVICE[i, j], 'food': FOOD[i, j]})
            TIP[i, j] = result['tip']
        except:
            TIP[i, j] = np.nan

fig = plt.figure(figsize=(16, 7))

# ============================================================================
# Subplot 1: 3D Surface
# ============================================================================
ax1 = fig.add_subplot(121, projection='3d')
surf = ax1.plot_surface(SERVICE, FOOD, TIP, cmap='viridis',
                        alpha=0.9, edgecolor='none', antialiased=True)
ax1.set_xlabel('Service Quality', fontsize=11, labelpad=10)
ax1.set_ylabel('Food Quality', fontsize=11, labelpad=10)
ax1.set_zlabel('Tip (%)', fontsize=11, labelpad=10)
ax1.set_title('Fuzzy Control Surface (3D)', fontsize=13, fontweight='bold', pad=15)
ax1.view_init(elev=25, azim=135)
fig.colorbar(surf, ax=ax1, shrink=0.5, aspect=5, label='Tip (%)')

# ============================================================================
# Subplot 2: Contour Map
# ============================================================================
ax2 = fig.add_subplot(122)
contour = ax2.contourf(SERVICE, FOOD, TIP, levels=20, cmap='viridis')
ax2.set_xlabel('Service Quality', fontsize=12)
ax2.set_ylabel('Food Quality', fontsize=12)
ax2.set_title('Control Surface (Contour)', fontsize=13, fontweight='bold')
fig.colorbar(contour, ax=ax2, label='Tip (%)')
ax2.grid(True, alpha=0.3, linestyle='--')

examples = [(2, 8), (8, 2), (5, 5), (9, 9), (1, 1)]
for s, f in examples:
    result = system.evaluate({'service': s, 'food': f})
    tip_val = result['tip']
    ax2.plot(s, f, 'ro', markersize=10, markeredgecolor='white', markeredgewidth=2)
    ax2.annotate(f'{tip_val:.1f}%', (s, f), xytext=(8, 8),
                textcoords='offset points', fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.4', facecolor='white',
                          edgecolor='black', alpha=0.8))

plt.tight_layout()
plt.show()